In [16]:
import numpy as np
import cv2 
import os
import imutils
import matplotlib.pyplot as plt
from PIL import Image
"""
This function is used to crop the images contained in the path directory and save it in the dest directory
It also displays the cropped images
"""
def crop(path,dest):

    if os.path.exists(dest):
        os.system('rm -r dest')
    if not os.path.exists(dest):
        os.makedirs(dest)
    if not os.path.exists(path):
        print("The given directory doesn't exist")
        return 
    process(path, dest)
    #show_images(dest)
    return

"""
This function is used to display the images contained in the directory
"""
def show_images(directory):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            if f.endswith('.ppm') or f.endswith('.jpeg') or f.endswith('.jp2'):
                cv2.imshow('image', cv2.imread(f))
                cv2.waitKey()
    return

"""
This function is used to process the images contained in the path directory and save it in the dest directory
"""
def process(path, dest):

    #----------------------------------------------------------------------------------------------------------

    directory = path
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if not os.path.isfile(f):
            print("Expected the directory to contain only files") 
        if os.path.isfile(f):
            if filename.endswith('.ppm') or filename.endswith('.jpeg') or filename.endswith('.jp2'):

    #----------------------------------------------------------------------------------------------------------

                print(f)
                raw_image = cv2.imread(f) #load the image
                process_image = raw_image.copy() #copy the image
                if(process_shape(raw_image, dest, filename, f)):
                    continue
                print(f)
    
    #----------------------------------------------------------------------------------------------------------
    # 3 

                # On floute avec du gaussian puis du median blur
                blur = cv2.GaussianBlur(process_image, (9,9), 0)
                blur = cv2.medianBlur(blur, 9)          

                # On transforme l'image en image HSV
                img_hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

                #Augmenter la saturation
                img_hsv[...,1] = img_hsv[...,1]*1.4

                # Définit la range du rouge en HSV
                lower_red = np.array([0, 50, 50])
                upper_red = np.array([10, 255, 255])

                # Crée un masque pour la couleur rouge en utilisant la range définie
                mask1 = cv2.inRange(img_hsv, lower_red, upper_red)
                # Masque rouge valeurs hautes
                lower_red = np.array([170, 70, 50])
                upper_red = np.array([180, 255, 255])
                mask_red = cv2.inRange(img_hsv.copy(), lower_red, upper_red)

                mask_red = cv2.bitwise_or(mask1, mask_red)

                #Masque orange 
                #lower_orange = np.array([5, 50, 50])
                #upper_orange = np.array([15, 255, 255])
                #mask_orange = cv2.inRange(img_hsv.copy(), lower_orange, upper_orange)

    #----------------------------------------------------------------------------------------------------------

                # Masque bleu
                lower_blue = np.array([90, 100, 100])
                upper_blue = np.array([121, 255, 255])
                mask_blue = cv2.inRange(img_hsv.copy(), lower_blue, upper_blue)

                #Total mask
                mask_red_blue = cv2.bitwise_or(mask_red, mask_blue) #mask orange 

                #show image with mask
                #cv2.imshow('mask', mask_red_blue)
                #cv2.waitKey()

    #----------------------------------------------------------------------------------------------------------

                # Trouve les contours dans l'image filtrée
                contours, _ = cv2.findContours(mask_red_blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                #draw image with contours
                #cv2.drawContours(process_image, contours, -1, (0, 255, 0), 3)
                #cv2.imshow('contour', process_image)
                #cv2.waitKey()

                # Initialise une liste vide pour stocker les contours valides
                list_contour = []
                best_contour = None

                # Parcourt tous les contours trouvés
                for cnt in contours:
                    # Calcule le rectangle englobant le contour
                    x, y, w, h = cv2.boundingRect(cnt)
                    #imposer que le bounding box soit plus grand que 5% de l'image 
                    
                    if (w*h) < (0.0005*raw_image.shape[0]*raw_image.shape[1]):
                        #print("condition 1")
                        continue

                    #imposer que le bounding box soit pas un rectangle très plat 
                    if (w/h) < 0.7 or (w/h) > 1.3:
                        #print("condition 2")
                        continue

                    #if it is a circle very likely to be a round road sign
                    area = cv2.contourArea(cnt)
                    perimeter = cv2.arcLength(cnt, True)
                    try:
                        circularity = 4 * np.pi * area / perimeter ** 2
                        if circularity > 0.8:
                            best_contour = cnt
                            list_contour.append(cnt)
                            break
                    except: 
                        pass 
                    #convexité ? 
                    try:
                        hull = cv2.convexHull(cnt)
                        hull_area = cv2.contourArea(hull)
                        solidity = float(area)/hull_area
                        if solidity < 0.7:
                            #print("condition 3")
                            continue
                    except: 
                        continue
                    
                    list_contour.append(cnt)

                #print(len(list_contour))

                after_contour = raw_image.copy()
                #cv2.drawContours(after_contour, list_contour, -1, (0, 255, 0), 3)
                #cv2.imshow('after contour', after_contour)
                #cv2.waitKey()

                #----------------------------------------------------------------------------------------------------------

                if(len(list_contour) == 0):
                    #print("No contour found")
                    continue
                    
                if(best_contour is None):
                    #get max of list_contour
                    best_contour = max(list_contour, key=lambda x: cv2.contourArea(x))

                #----------------------------------------------------------------------------------------------------------

                (x,y,w,h) = cv2.boundingRect(best_contour)
                #cv2.rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
                #cv2.drawContours(raw_image, [best_contour],  -1, (255,0,0), 2)
                cv2.waitKey()
                raw_image = raw_image[y:y + h, x:x + w]
                img = cv2.imread(f)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)   
                cv2.imwrite(dest+"/"+filename, raw_image)

    return 


def process_shape(raw_image, dest, filename, f):

    #----------------------------------------------------------------------------------------------------------
    # 1

    process_image = raw_image.copy()
    gray = cv2.cvtColor(process_image, cv2.COLOR_BGR2GRAY)
    #cv2.equalizeHist(gray) 
    bilateral_filtered_image = cv2.GaussianBlur(gray,(3,3),0)
    edge_detected_image = cv2.Canny(bilateral_filtered_image, 90, 170)

    #----------------------------------------------------------------------------------------------------------
    # 2
    
    # Convert image to HSV color space
    hsv = cv2.cvtColor(process_image, cv2.COLOR_BGR2HSV)
    lower_sat = np.array([50, 50, 50]) # Minimum saturation and value thresholds
    upper_sat = np.array([255, 255, 255])
    mask = cv2.inRange(hsv, lower_sat, upper_sat)

    #----------------------------------------------------------------------------------------------------------
    
    # Apply mask to image to remove grayscale regions
    masked_image = cv2.bitwise_and(process_image, process_image, mask=mask)

    #----------------------------------------------------------------------------------------------------------
    # 1 

    gray = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)
    bilateral_filtered_image = cv2.GaussianBlur(gray,(3,3),0)
    edge_detected_image = cv2.Canny(bilateral_filtered_image, 90, 170)

    #----------------------------------------------------------------------------------------------------------
    # 2

    contours = cv2.findContours(edge_detected_image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #argument cv2.CHAIN_APPROX_SIMPLE ? instead of cv2.CHAIN_APPROX_NONE
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:20]

    #----------------------------------------------------------------------------------------------------------

    shapes = []
    # Loop over contours and categorize by shape
    for contour in contours:

    #----------------------------------------------------------------------------------------------------------
    # A function to give that 

        length = cv2.arcLength(contour,True)
        approx = cv2.approxPolyDP(contour,0.01*length, True)
        # Check for regularity of sides
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        if(perimeter == 0):
            continue
        circularity = 4 * np.pi * area / perimeter ** 2

    #----------------------------------------------------------------------------------------------------------
    # A lot of conditions 
    
        if circularity < 0.8:
            continue
        if not cv2.isContourConvex(approx):
            continue
        if len(approx) == 3:
            shapes.append(contour)
        elif len(approx) == 4:
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            if (ar >= 0.8 and ar <= 1.2):
                shapes.append(contour)
            else:
                continue
        elif len(approx) == 8:
            shapes.append(contour)

        elif len(approx) > 8: # check if contour has a circular shape
            (x, y), radius = cv2.minEnclosingCircle(contour)
            area = cv2.contourArea(contour)
            if radius > 0 and area / (np.pi * radius**2) >= 0.8:
                shapes.append(contour)
        else:
            continue

    #----------------------------------------------------------------------------------------------------------

    # Find contours with largest areas for each shape
    try:
        best_contour = max(shapes, key=lambda x: cv2.contourArea(x))
    except ValueError:
        best_contour = None
    if best_contour is None:
        print('No shape found')
        #bounding box is the whole image
        #cv2.imwrite(dest + "/"+filename, raw_image)
        return False

    #----------------------------------------------------------------------------------------------------------
    
    (x,y,w,h) = cv2.boundingRect(best_contour)
    #cv2.rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
    #cv2.drawContours(raw_image, [best_contour],  -1, (255,0,0), 2)
    cv2.waitKey()
    raw_image = raw_image[y:y + h, x:x + w]
    img = cv2.imread(f)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)   

    cv2.imwrite(dest + "/"+filename, raw_image)
    return True

    #----------------------------------------------------------------------------------------------------------
    
def main():
    crop('challenge-2/eval_kaggle2', 'result_cropped_eval2')
    return
if __name__ == "__main__":
    main()

challenge-2/eval_kaggle2\1.ppm
No shape found
challenge-2/eval_kaggle2\1.ppm
challenge-2/eval_kaggle2\10.ppm
No shape found
challenge-2/eval_kaggle2\10.ppm
challenge-2/eval_kaggle2\11.ppm
No shape found
challenge-2/eval_kaggle2\11.ppm
challenge-2/eval_kaggle2\12.ppm
No shape found
challenge-2/eval_kaggle2\12.ppm
challenge-2/eval_kaggle2\13.ppm
No shape found
challenge-2/eval_kaggle2\13.ppm
challenge-2/eval_kaggle2\14.ppm
No shape found
challenge-2/eval_kaggle2\14.ppm
challenge-2/eval_kaggle2\15.ppm
No shape found
challenge-2/eval_kaggle2\15.ppm
challenge-2/eval_kaggle2\16.ppm
challenge-2/eval_kaggle2\17.ppm
No shape found
challenge-2/eval_kaggle2\17.ppm
challenge-2/eval_kaggle2\18.ppm
No shape found
challenge-2/eval_kaggle2\18.ppm
challenge-2/eval_kaggle2\19.ppm
No shape found
challenge-2/eval_kaggle2\19.ppm
challenge-2/eval_kaggle2\2.ppm
No shape found
challenge-2/eval_kaggle2\2.ppm
challenge-2/eval_kaggle2\20.ppm
No shape found
challenge-2/eval_kaggle2\20.ppm
challenge-2/eval_kaggle2